In [ ]:
## 导入文件
with open("techFilt_left.vcf","r") as f:
    line_list=f.readlines()
    num=0
    ref=[]
    alt=[]
    chrom=[]
    pos=[]
    genotype=[]
    for i in range(len(line_list)):
        tmp=line_list[i].split()
        chrom.append(tmp[0])
        pos.append(tmp[1])
        ref.append((tmp[3]).split(","))
        alt.append((tmp[4]).split(","))
        ## 取出所有个体的基因型，填入一个二维列表中
        genotype.append([])
        for j in range(9,61):
            tmp2=tmp[j].split(":")
            genotype[num].append(tmp2[0])
        num+=1

In [12]:
## 标记二等位的snp位点(在这一步去掉indel)
count=0
allele=[]
for i in range(len(alt)):
    if len(alt[i])==1 and len(alt[i][0])==1 and len(ref[i][0])==1:
        count+=1
        allele.append(0)
    else:
        allele.append(99)
print(count/len(alt))

0.8774116076197258


In [13]:
#标记母本基因型是否被检出
mtl_exi=[]
for i in range(len(chrom)):
    if genotype[i][0]=='./.' or genotype[i][0]=='.|.':
        mtl_exi.append(99)
    else:
        mtl_exi.append(0)

#计数子代各种基因型的数量
off_exi=[]
off_zero_zero=[]
off_zero_one=[]
off_one_one=[]
for i in range(len(chrom)):
    null=0
    zero_zero=0
    zero_one=0
    one_one=0
    for j in range(1,52):
        if genotype[i][j]=='./.' or genotype[i][j]=='.|.':
            null+=1
        elif genotype[i][j]=='0/0' or genotype[i][j]=='0|0':
            zero_zero+=1
        elif genotype[i][j]=='0/1' or genotype[i][j]=='0|1':
            zero_one+=1
        elif genotype[i][j]=='1/1' or genotype[i][j]=='1|1':
            one_one+=1    
    off_exi.append(51-null)
    off_zero_zero.append(zero_zero)
    off_zero_one.append(zero_one)
    off_one_one.append(one_one)

In [15]:
# 标记符合孟德尔分离定律的位点，标为0，其余为99.
count=0
seg_test=[]
for i in range(len(chrom)):
    if allele[i]==0 and \
    mtl_exi[i]==0 and \
    (((not off_zero_zero[i]==0) and off_one_one[i]==0) or \
     ((not off_one_one[i]==0) and off_zero_zero[i]==0)):
    ### 0/0 和 1/1必须有其一是0，不然有可能是自交的情况
            
        if off_exi[i]==51:
            if off_zero_one[i]>=19 and off_zero_one[i]<=32:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        elif off_exi[i]==50:
            if off_zero_one[i]>=19 and off_zero_one[i]<=31:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        elif off_exi[i]==49:
            if off_zero_one[i]>=18 and off_zero_one[i]<=31:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        elif off_exi[i]==48:
            if off_zero_one[i]>=18 and off_zero_one[i]<=30:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        else:
            seg_test.append(99)
    else:
        seg_test.append(99)
print(count,count/len(chrom),count/len(alt))

1696697 0.3020646252447926 0.3020646252447926


In [16]:
for i in range(744,746):
    print(chrom[i],pos[i],mtl_exi[i],off_exi[i],off_zero_zero[i],off_zero_one[i],off_one_one[i],seg_test[i])

scaffold1 49987 0 51 22 29 0 99
scaffold1 50067 0 51 27 24 0 0


In [18]:
# 硬删除
with open("techFilt_left.vcf","r") as f:
    line_list=f.readlines()
with open("bioFilt_left.vcf","w") as g:
    g.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tmaternal1"\
            +"\toff05\toff06\toff07\toff_L1_80\toff_L1_81\toff_L1_83\toff_L2_85"\
            +"\toff_L2_86\toff_L2_89\toff_L3_90\toff_L3_92\toff_L3_93\toff_L4_94"\
            +"\toff_L4_96\toff_L4_97\toff_L5_100\toff_L5_101\toff_L5_98\toff_L6_102"\
            +"\toff_L6_106\toff_L6_108\toff_L7_110\toff_L7_111\toff_L7_112\toff_L8_103"\
            +"\toff_L8_113\toff_L8_95\txL1_58\txL1_59\txL1_60\txL2_61\txL2_62\txL2_63"\
            +"\txL3_66\txL3_67\txL3_68\txL4_70\txL4_71\txL4_72\txL5_73\txL5_75\txL5_76"\
            +"\txL6_77\txL6_78\txL6_79\txL7_80\txL7_81\txL7_83\txL8_84\txL8_85\txL8_86\n")
    for i in range(len(line_list)):
            if seg_test[i]==0:
                g.write(line_list[i])
    g.close()
    maternal1       off05      off06   off07   off_L1_80
    off_L1_81       off_L1_83       off_L2_85       off_L2_86  off_L2_89       
    off_L3_90       off_L3_92       off_L3_93       off_L4_94       off_L4_96  
    off_L4_97       off_L5_100      off_L5_101      off_L5_98       off_L6_102      
    off_L6_106 off_L6_108      off_L7_110      off_L7_111      off_L7_112      
    off_L8_103      off_L8_113 off_L8_95       xL1_58  xL1_59  
    xL1_60  xL2_61  xL2_62  xL2_63  xL3_66  
    xL3_67  xL3_68     xL4_70  xL4_71  xL4_72  
    xL5_73  xL5_75  xL5_76  xL6_77  xL6_78  
    xL6_79  xL7_80  xL7_81     xL7_83  xL8_84  xL8_85  xL8_86

In [19]:
# 取前50kb
with open("techFilt_left.vcf","r") as f:
    line_list=f.readlines()[:745]
with open("bioFilt_left_head50kb.vcf","w") as g:
    g.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tmaternal1"\
            +"\toff05\toff06\toff07\toff_L1_80\toff_L1_81\toff_L1_83\toff_L2_85"\
            +"\toff_L2_86\toff_L2_89\toff_L3_90\toff_L3_92\toff_L3_93\toff_L4_94"\
            +"\toff_L4_96\toff_L4_97\toff_L5_100\toff_L5_101\toff_L5_98\toff_L6_102"\
            +"\toff_L6_106\toff_L6_108\toff_L7_110\toff_L7_111\toff_L7_112\toff_L8_103"\
            +"\toff_L8_113\toff_L8_95\txL1_58\txL1_59\txL1_60\txL2_61\txL2_62\txL2_63"\
            +"\txL3_66\txL3_67\txL3_68\txL4_70\txL4_71\txL4_72\txL5_73\txL5_75\txL5_76"\
            +"\txL6_77\txL6_78\txL6_79\txL7_80\txL7_81\txL7_83\txL8_84\txL8_85\txL8_86\n")
    for i in range(len(line_list)):
            if seg_test[i]==0:
                g.write(line_list[i])
    g.close()

In [ ]:
# 根据符合孟德尔分离定律的二等位位点预测副本基因型